In [1]:
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions, ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import splitfolders
import numpy as np

In [2]:
#take data and randomly distribute them (remove bias with chance)
path = '/Users/rajatdoshi/Desktop/excema_classifier/dataset'
output = '/Users/rajatdoshi/Desktop/excema_classifier/processed_data'
splitfolders.ratio(path, output, seed=42, ratio=(.6,.2,.2))

Copying files: 1554 files [00:00, 2683.47 files/s]


In [3]:
img_height, img_width = (224,224)
batch_size = 32

train_data_dir = r'/Users/rajatdoshi/Desktop/excema_classifier/processed_data/train'
valid_data_dir = r'/Users/rajatdoshi/Desktop/excema_classifier/processed_data/val'
test_data_dir = r'/Users/rajatdoshi/Desktop/excema_classifier/processed_data/test'

In [4]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  validation_split=0.4)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=1,
    class_mode='categorical',
    subset='training')

valid_generator = train_datagen.flow_from_directory(
    valid_data_dir,
    target_size=(img_height, img_width),
    batch_size=1,
    class_mode='categorical',
    subset='validation')

test_generator = train_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=1,
    class_mode='categorical',
    subset='validation')

Found 560 images belonging to 6 classes.
Found 121 images belonging to 6 classes.
Found 123 images belonging to 6 classes.


In [5]:
base_model = ResNet50(include_top=False, weights='imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x)
x= Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_generator, epochs=2)

Epoch 1/2
560/560 [==============================] - 56s 96ms/step - loss: 3.1541 - accuracy: 0.2709
Epoch 2/2
560/560 [==============================] - 51s 91ms/step - loss: 1.4721 - accuracy: 0.4129


In [6]:
model.save('/Users/rajatdoshi/Desktop/tf/eczemaclassifier.h5')

In [7]:
test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print('\nTest Accuracy: ', test_acc)

123/123 [==============================] - 11s 84ms/step - loss: 1.3700 - accuracy: 0.4146

Test Accuracy:  0.4146341383457184
